In [1]:
import itertools

In [2]:
spells = {
    "magic_missile": {
        "mana_cost": 53,
        "damage": 4,
    },
    "drain": {
        "mana_cost": 73,
        "damage": 2,
        "heals": 2
    },
    "shield": {
        "mana_cost": 113,
        "armor": 8,
        "lasts": 6
    },
    "poison": {
        "mana_cost": 173,
        "damage": 3,
        "lasts": 6
    },
    "recharge": {
        "mana_cost": 229,
        "mana_gain": 101,
        "lasts": 5
    },
}

In [3]:
day22_inputs = "inputs/day22.txt"
with open(day22_inputs, 'r') as file:
    day22_data = file.readlines()
day22_data_split = [el.strip().split(": ") for el in day22_data]
day22_data_flat = list(itertools.chain.from_iterable(day22_data_split))

boss = {day22_data_flat[i]: int(day22_data_flat[i + 1]) for i in range(0, len(day22_data_flat), 2)}

print(boss)

{'Hit Points': 55, 'Damage': 8}


In [4]:
def player_effects(active_effects, curr_player_mana_points, curr_player_armor, curr_boss_hit_points):
    # make a copy so we don't mutate caller
    new_effects = active_effects.copy()
    
    for effect in list(new_effects.keys()):
        turns_left = new_effects[effect]
        new_turns_left = turns_left - 1
        new_effects[effect] = new_turns_left

        if effect == "recharge":
            curr_player_mana_points += spells[effect]["mana_gain"]
            # print(f"{effect} provides mana, player has {curr_player_mana_points} mana points; effect timer is now {new_turns_left}")
        if effect == "poison":
            curr_boss_hit_points -= spells[effect]["damage"]
            # print(f"{effect} deals damage, boss has {curr_boss_hit_points}; effect timer is now {new_turns_left}")
        if new_turns_left == 0:
            if effect == "shield":
                curr_player_armor = 0
            del new_effects[effect]
            # print(f"{effect} has worn off")

    return {
        "player_mana": curr_player_mana_points,
        "player_armor": curr_player_armor,
        "boss_hp": curr_boss_hit_points,
        "effects": new_effects
    }


def player_turn(spells, spell, curr_boss_hit_points, curr_player_mana_points, curr_player_armor, curr_player_hit_points, curr_turn_player_damage, active_effects):
    spell_mana_cost = spells[spell]["mana_cost"]

    if spell_mana_cost > curr_player_mana_points:
        # cannot cast spell
        return None

    curr_player_mana_points -= spell_mana_cost
    new_effects = dict(active_effects)

    # print(f"player casts {spell} for {spell_mana_cost} and has {curr_player_mana_points} mana remaining")

    if "lasts" in spells[spell]:  # it's an effect spell
        spell_lasts = spells[spell]["lasts"]
        new_effects.setdefault(spell, spell_lasts)
        if spell == "shield":
            curr_player_armor = 7
    else:  # instant effect
        curr_boss_hit_points -= spells[spell].get("damage", 0)
        curr_turn_player_damage += spells[spell].get("damage", 0)
        if spell == "drain":
            curr_player_hit_points += spells[spell].get("heals", 0)

    return {
        "player_hp": curr_player_hit_points,
        "player_mana": curr_player_mana_points,
        "player_armor": curr_player_armor,
        "boss_hp": curr_boss_hit_points,
        "effects": new_effects
    }


In [7]:
player = {
    "Hit Points": 50,
    "Mana Points": 500,
    "Damage": 0,
    "Armor": 0,
    "Effects": {}
}

curr_boss_hit_points = boss["Hit Points"]

curr_player_hit_points = player["Hit Points"]
curr_player_mana_points = player["Mana Points"]

curr_player_armor = player["Armor"]
curr_player_damage = player["Damage"]

def search(curr_player_hit_points, curr_player_mana_points, curr_player_armor,
           curr_player_active_effects, curr_boss_hit_points, turn, mana_spent):
    
    global best_mana
    
    # apply active effects at the start of the turn
    result = player_effects(dict(curr_player_active_effects),
                            curr_player_mana_points,
                            curr_player_armor,
                            curr_boss_hit_points)
    
    curr_player_active_effects = tuple(result["effects"].items())
    curr_player_mana_points = result["player_mana"]
    curr_player_armor = result["player_armor"]
    curr_boss_hit_points = result["boss_hp"]
    
    # check for boss death after effects
    if curr_boss_hit_points <= 0:
        old_best = best_mana
        best_mana = min(best_mana, mana_spent)
        if best_mana < old_best:
            print(f"best mana is {best_mana} so far")
        return

    
    # check for player death
    if curr_player_hit_points <= 0:
        return
    
    if turn == "player":
        # try every legal spell
        for spell_name, spell_data in spells.items():
            # cannot cast if not enough mana or effect is active
            if spell_data["mana_cost"] > curr_player_mana_points:
                continue
            if spell_name in dict(curr_player_active_effects):
                continue
            
            # apply the spell
            result = player_turn(spells, spell_name,
                                 curr_boss_hit_points,
                                 curr_player_mana_points,
                                 curr_player_armor,
                                 curr_player_hit_points,
                                 0,
                                 dict(curr_player_active_effects))
            
            # skip spells that could not be cast
            if result is None:
                continue
            
            # recurse to boss turn
            search(result["player_hp"],
                   result["player_mana"],
                   result["player_armor"],
                   tuple(result["effects"].items()),
                   result["boss_hp"],
                   "boss",
                   mana_spent + spells[spell_name]["mana_cost"])
    
    else:  # boss turn
        boss_damage = max(1, boss["Damage"] - curr_player_armor)
        next_player_hp = curr_player_hit_points - boss_damage
        
        # recurse to player turn
        search(next_player_hp,
               curr_player_mana_points,
               curr_player_armor,
               curr_player_active_effects,
               curr_boss_hit_points,
               "player",
               mana_spent)

In [8]:
best_mana = float("inf")

search(
    player["Hit Points"],
    player["Mana Points"],
    player["Armor"],
    tuple(),
    boss["Hit Points"],
    "player",
    0
)

print(best_mana)


best mana is 2971 so far
best mana is 2845 so far
best mana is 2483 so far
best mana is 2297 so far
best mana is 2002 so far
best mana is 1929 so far
best mana is 1743 so far
best mana is 1461 so far
best mana is 1348 so far
